# Student Progress Monitoring System

In [2]:
import os
import importlib
import ipywidgets as widgets
from IPython.display import display, clear_output

out = widgets.Output(layout={"border": "1px solid #ddd", "padding": "10px"})
status = widgets.HTML(value="<b>Status:</b> Ready")

# -----------------------------------------------------------------------------
# User inputs (manual paths).
# -----------------------------------------------------------------------------
csv_folder = widgets.Text(
    value=r"",
    description="CSV folder:",
    placeholder=r"e.g. D:\CW\CWdatafiles",
    layout=widgets.Layout(width="900px"),
)

db_path = widgets.Text(
    value=r"/CWDatabase.db",
    description="DB path:",
    placeholder=r"e.g. D:\Course_Work\CWDatabase.db",
    layout=widgets.Layout(width="900px"),
)

csv_files = widgets.Textarea(
    value="",
    description="CSV files:",
    placeholder="Optional: one full CSV path per line (if provided, folder is ignored).",
    layout=widgets.Layout(width="900px", height="120px"),
)

btn_load = widgets.Button(description="Load modules", button_style="info")
btn_build_db = widgets.Button(description="Convert CSVs → SQLite", button_style="success")

# -----------------------------------------------------------------------------
# Tabs: Student Results / Per-Question / Underperformers.
# -----------------------------------------------------------------------------
student_id_all = widgets.Text(
    value="",
    description="Student ID:",
    placeholder="e.g. 156",
    layout=widgets.Layout(width="400px"),
)
btn_all_results = widgets.Button(description="Show all results", button_style="primary")

student_id_q = widgets.Text(
    value="",
    description="Student ID:",
    placeholder="e.g. 156",
    layout=widgets.Layout(width="400px"),
)
test_name_q = widgets.Dropdown(
    options=["Test_1", "Test_2", "Test_3", "Test_4", "Mock_Test", "Sum_Test"],
    value="Test_1",
    description="Test:",
    layout=widgets.Layout(width="400px"),
)
btn_question = widgets.Button(description="Analyse per-question", button_style="primary")

btn_under = widgets.Button(description="Identify underperformers", button_style="warning")

# -----------------------------------------------------------------------------
# Module state.
# -----------------------------------------------------------------------------
STATE = {"loaded": False, "mods": {}}

def _set_status(msg: str):
    status.value = f"<b>Status:</b> {msg}"

def _resolve_csv_list():
    """Build a list of CSV paths from the textarea or the folder."""
    lines = [ln.strip() for ln in csv_files.value.splitlines() if ln.strip()]
    if lines:
        return lines

    folder = csv_folder.value.strip()
    if not folder:
        raise ValueError("Please enter a CSV folder path.")
    if not os.path.isdir(folder):
        raise FileNotFoundError(f"Folder not found: {folder}")

    expected = [
        "Formative_Test_1.csv",
        "Formative_Test_2.csv",
        "Formative_Test_3.csv",
        "Formative_Test_4.csv",
        "Formative_Mock_Test.csv",
        "StudentRate.csv",
        "SumTest.csv",
    ]
    paths = [os.path.join(folder, name) for name in expected]
    missing = [p for p in paths if not os.path.exists(p)]
    if missing:
        raise FileNotFoundError("Missing CSV file(s):\n" + "\n".join(missing))
    return paths

def load_modules(_=None):
    """Loads/reloads modules so the notebook uses the latest code."""
    with out:
        clear_output()
        _set_status("Loading modules...")

        import CWPreprocessing
        import studentPerformance
        import underperformingStudent
        import testResults

        importlib.reload(CWPreprocessing)
        importlib.reload(studentPerformance)
        importlib.reload(underperformingStudent)
        importlib.reload(testResults)

        STATE["mods"] = {
            "CWPreprocessing": CWPreprocessing,
            "studentPerformance": studentPerformance,
            "underperformingStudent": underperformingStudent,
            "testResults": testResults,
        }
        STATE["loaded"] = True
        _set_status("Import complete. Refresh table list in analysis tabs if needed.")
        print("Modules loaded OK.")

def build_database(_=None):
    """Builds SQLite tables from the CSV files."""
    with out:
        _set_status("Building database...")
        if not STATE["loaded"]:
            print("Please click 'Load modules' first.")
            return

        db = db_path.value.strip()
        if not db:
            raise ValueError("Please enter a DB path.")

        paths = _resolve_csv_list()

        CWPreprocessor = STATE["mods"]["CWPreprocessing"].CWPreprocessor
        by_name = {os.path.basename(p): p for p in paths}

        pre = CWPreprocessor(
            db_path=db,
            file_test_1=by_name["Formative_Test_1.csv"],
            file_test_2=by_name["Formative_Test_2.csv"],
            file_test_3=by_name["Formative_Test_3.csv"],
            file_test_4=by_name["Formative_Test_4.csv"],
            file_mock=by_name["Formative_Mock_Test.csv"],
            file_rate=by_name["StudentRate.csv"],
            file_sum=by_name["SumTest.csv"],
        )
        pre.run()
        print("Database written successfully:", db)
        _set_status("Database build finished.")

def show_all_results(_=None):
    """Shows all results for a student."""
    with out:
        _set_status("Running...")
        if not STATE["loaded"]:
            print("Please click 'Load modules' first.")
            return
        sid = student_id_all.value.strip()
        if not sid:
            raise ValueError("Please enter a Student ID.")
        db = db_path.value.strip()
        if not db:
            raise ValueError("Please enter a DB path.")

        fetch_all_results = STATE["mods"]["testResults"].fetch_all_results
        fetch_all_results(db, sid)
        _set_status("Done.")

def analyse_per_question(_=None):
    """Runs per-question analysis for one student and one test."""
    with out:
        _set_status("Running...")
        if not STATE["loaded"]:
            print("Please click 'Load modules' first.")
            return
        sid = student_id_q.value.strip()
        if not sid:
            raise ValueError("Please enter a Student ID.")
        db = db_path.value.strip()
        if not db:
            raise ValueError("Please enter a DB path.")

        student_performance = STATE["mods"]["studentPerformance"].student_performance
        student_performance(db, int(sid), test_name_q.value)
        _set_status("Done.")

def run_underperformers(_=None):
    """Finds underperforming students and plots the chart."""
    with out:
        _set_status("Running...")
        if not STATE["loaded"]:
            print("Please click 'Load modules' first.")
            return
        db = db_path.value.strip()
        if not db:
            raise ValueError("Please enter a DB path.")

        find_underperforming_students = STATE["mods"]["underperformingStudent"].find_underperforming_students
        find_underperforming_students(db)
        _set_status("Done.")

btn_load.on_click(load_modules)
btn_build_db.on_click(build_database)
btn_all_results.on_click(show_all_results)
btn_question.on_click(analyse_per_question)
btn_under.on_click(run_underperformers)

tab_csv = widgets.VBox([
    widgets.HTML("<b>1) Import CSV files into SQLite</b><br>Provide a folder OR list CSV files manually."),
    csv_folder,
    csv_files,
    widgets.HBox([btn_build_db]),
])

tab_student = widgets.VBox([
    widgets.HTML("<b>2) Student Results</b><br>Show all test results for one student."),
    student_id_all,
    btn_all_results,
])

tab_question = widgets.VBox([
    widgets.HTML("<b>3) Per-Question Analysis</b><br>Analyse question-level performance for one test."),
    widgets.HBox([student_id_q, test_name_q]),
    btn_question,
])

tab_under = widgets.VBox([
    widgets.HTML("<b>4) Underperformers</b><br>Identify underperforming students and plot comparison."),
    btn_under,
])

tabs = widgets.Tab(children=[tab_csv, tab_student, tab_question, tab_under])
tabs.set_title(0, "CSV → SQLite")
tabs.set_title(1, "Student Results")
tabs.set_title(2, "Per-Question Analysis")
tabs.set_title(3, "Underperformers")

ui = widgets.VBox([
    widgets.HTML("<h2>Menu Toolkit</h2>"),
    widgets.HBox([btn_load, status]),
    db_path,
    tabs,
    widgets.HTML("<b>Output / Logs</b>"),
    out
])

display(ui)
